This is the notebook dedicated to running our models

Gather the dataset and imports

In [1]:
import sys
import os
import torch
import torch.optim as optim
import torch.nn as nn
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from data.data_preprocessing import MRIDataset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import pickle

# Applying random transformations to vary data
transformations = transforms.Compose([
    transforms.RandomRotation(15),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip()
])

# Data preloaded from pickle file
data_path = os.path.abspath(os.path.join("..", "data", "archive", "brain_tumor_mri", "new_dataset", "training_data.pickle"))
with open(data_path, 'rb') as file:
    loaded_data = pickle.load(file)

# Unpacking the data into the images and their corresponding labels
images, labels = zip(*loaded_data)
dataset = MRIDataset(images, labels, transformations, model_type=None)

# Splitting the dataset into training and testing
training_size = int(.8 * len(dataset))
testing_size = len(dataset) - training_size
training_dataset, testing_dataset = random_split(dataset, [training_size, testing_size])

# Two separate loaders for training and testing
train_loader = DataLoader(training_dataset, batch_size=16, shuffle=True)
testing_loader = DataLoader(testing_dataset, batch_size=16, shuffle=True)

# Debug comment out later
for sample_image, sample_label in train_loader:
    print(f"Image shape: {sample_image.shape}")
    print(f"Label: {sample_label}")
    break


/usr/local/Caskroom/miniconda/base/envs/TumorTrace/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/usr/local/Caskroom/miniconda/base/envs/TumorTrace/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <E03EDA44-89AE-3115-9796-62BA9E0E2EDE> /usr/local/Caskroom/miniconda/base/envs/TumorTrace/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <ADD14A6E-669F-3F53-A802-D8FBD6AA3F40> /usr/local/Caskroom/miniconda/base/envs/TumorTrace/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


Image shape: torch.Size([16, 1, 512, 512, 3])
Label: tensor([3., 1., 1., 2., 3., 2., 3., 2., 2., 3., 2., 3., 2., 2., 3., 1.])


Run the 3D CNN model

In [ ]:
from models.CNN_3D.model import CNN_3D
from models.CNN_3D.train import Trainer

# Set up the model, criterion, and optimizer
model = CNN_3D(num_classes=3)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Initialize the Trainer class
trainer = Trainer(
    model=model,
    train_loader=train_loader,
    val_loader=testing_loader,
    criterion=criterion,
    optimizer=optimizer,
    num_epochs=10,
    device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
)

trainer.train()

Batch number 0, Loss: 1.0783663988113403
Batch number 1, Loss: 445.1538391113281
Batch number 2, Loss: 798.89306640625
Batch number 3, Loss: 123.47567749023438
Batch number 4, Loss: 36.87293243408203
Batch number 5, Loss: 27.177471160888672
Batch number 6, Loss: 39.0335807800293
Batch number 7, Loss: 14.636894226074219
Batch number 8, Loss: 9.445398330688477
Batch number 9, Loss: 18.422515869140625
Batch number 10, Loss: 7.320761680603027
Batch number 11, Loss: 5.22018575668335
Batch number 12, Loss: 1.8114267587661743
Batch number 13, Loss: 1.2464625835418701
Batch number 14, Loss: 1.4600677490234375


Run the ResNet model

In [ ]:
from models.ResNet.model import ResNet
from models.ResNet.train import train

model = ResNet()

Run the Inception model

In [ ]:
from models.Inception.model import Inception
from models.Inception.train import train

model = Inception()

Visualize Models

In [ ]:
trainer.visualizer.plot_loss()
trainer.visualizer.plot_accuracy()
# Add ability to visualize confusion matrix probably